# Aqarios Luna Platform Demo

This jupyter notebook will walk you through some of the potential uses of the Luna platform with the goal of demonstrating how this tool can be integrated into existing workflows. 

This demo will only include functionality from LunaSolve, our platform focused on optimization using state-of-the-art quantum, hybrid, and classical heuristics.

The structure of this notebook is as follows:
 - **Part 1**: Introduction
 - **Part 2**: LunaSolve
 - **Part 3**: Setup
 - **Part 4**: Real-world optimization using built-in transformations
 - **Part 5**: Open-ended optimization using manually constructed QUBOs

For a complete overview of our API's functionality visit https://api.aqarios.com/docs for interative documentation.

## Part 1: Introduction
Many business problems that companies face today can be formulated as optimization problems, whether in supply chain management, portfolio optimization, or balancing demand and supply for electricity. However, many of our real-world optimization problems are so complex that they cannot be solved efficiently or at all with our current hardware - *classical computers*. This leads to extremely inefficient workflows throughout our economy and huge losses that can even run into the billions for larger companies.

This is exactly where **Quantum Computing** will come into their own. The huge leap in computing power and speed will allow us to efficiently solve these optimization problems, leading to unimaginable advances in the future of our economy.

Aqarios Luna is a platform that will enable businesses to benefit from quantum technology and solve the most complex business problems. While quantum computers have tremendous potential, the complexity of communicating with them as well as understanding and applying the algorithms is a major hurdle to overcome and one that many companies struggle with. This is where we come in - Luna provides an intuitive and easy-to-use interface for accessing and applying quantum computing from the perspective of a business user - the use case and its solution. With Luna, there is no need to understand how quantum algorithms work in order to apply them. At the same time, Luna offers full customization and flexibility for those users who have the necessary knowledge and want to delve deeper.


## Part 2: LunaSolve

The service you will learn about in this notebook is **LunaSolve**, our service for solving optimization problems efficiently. LunaSolve offers 3 main functions:

- **Transform**: You can provide us with your use case in a predefined but intuitive and easy to understand format, which we will automatically transform into the necessary format to solve the problem on quantum devices.
- **Recommend**: We give you a recommendation on the best algorithm and hardware to use to solve your specific problem.
- **Solve**: You tell us which solver you want to use - we take care of everything necessary to get the algorithm onto quantum hardware and solve your problem.

Note that our platform automatically saves your problem formulations and generated solutions, so you can access them again at any later time without having to resubmit the optimization problem or run the solver again.

## Part 3: Setup

In order to make this notebook interactive, there are a few necessary steps to setup our environment.

### A. Install the necessary packages

In [ ]:
%pip install requests
%pip install networkx

### B. Set the API URL for requests

In [17]:
# URL = "https://api.aqarios.com"
URL = "http://0.0.0.0:8000"

### C. Authenticate your account

Create a file in this same directory named: `credentials.json`.

This will house your secret login information so that you can request access tokens from the platform.

The file should look like this:

```
{
    "username": "YOUREMAILHERE",
    "password": "YOURPASSWORDHERE"
}
```

In [15]:
import json
import requests

In [87]:
# get credentials from secret file
with open("credentials.json", 'r') as f:
    credentials = json.load(f)
    username = credentials['username']
    password = credentials['password']

# request API for token using credentials
response = requests.post(
    url=URL+"/accessToken",
    data={
        "username": username,
        "password": password
    }
)

# set received token as header
token = response.json().get("access_token")
headers = {"Authorization": f"Bearer {token}"}

## Part 4: Real-world optimization

There are many computationally challenging optimization problems defined in the literature, which cover a variety of domains. Examples like the Traveling Salesman Problem, Job-Shop Scheduling, or the Knapsack Problem are likely familiar to many who have spent time in the field. 

Many, but not all, of these optimization problems have been analized by researchers interested in attempting to solve them of quantum hardware. This process requires a transformative step. 

While the Traveling Salesman Problem is described by a graph which contains all the desired destinations and the distances between them, this representation is not suitable for current quantum annealer, let alone classical optimization heuristics. 

This is the first functionaly of LunaSolve: **Transform**. 

We provide users with a built-in library of (currently) **40** different optimization problems which can be configured using human-readable data formats (such as graphs). These are then automatically transformed into appropriate mathematical represetations for quantum computers.

### A. Retrieve available optimizations

The list of currently available optimizations can be retrieved using the `/api/problem/info/all` endpoint. However, we recommend browsing this information using the interactive documentation referenced above.

Here is an example of an optimization specification from the docs:

<div>
<img src="static/tsp-docs.jpg" width="800"/>
</div>

This includes a description of the optimization problem, its attributes, and references to the publication that its transformation is taken from.

The full list can be found by:
1. Navigating to https://api.aqarios.com/docs#/Optimizations/create_optimization_api_optimizations_post
2. Clicking on **Schema** in the Request Body section.
3. Opening the `instance` dropdown.
4. Opening the optimization problem of your choice.


### B. Create an instance of an optimization

Suppose you need to coordinate the shipping of wholesale products to across Germany. You have retailers in the seven largest German cites and want to deliver all of your product at once using the shortest route possible to minize transportation costs.

After browsing our list of available optimization problems, you notice that we have already implemented the Travelling Salesman problem. After reviewing the schema of the formulation in the manner described above, you find that the problem can be referenced by the *name* TSP and you see that the problem can be defined by a graph with edge weights representing the distances between each major city.

In [ ]:
import networkx as nx

In [ ]:
# Create an empty graph
graph = nx.Graph()

# Add nodes
cities = ['Berlin', 'Hamburg', 'Munich', 'Cologne', 'Frankfurt', 'Stuttgart', 'Düsseldorf']
graph.add_nodes_from(cities)

# Add edges and their weights (distances between cities)
edges = [('Berlin', 'Hamburg', 289), 
         ('Berlin', 'Munich', 590), 
         ('Berlin', 'Cologne', 579), 
         ('Berlin', 'Frankfurt', 551), 
         ('Berlin', 'Stuttgart', 639), 
         ('Berlin', 'Düsseldorf', 569), 
         ('Hamburg', 'Munich', 791), 
         ('Hamburg', 'Cologne', 433), 
         ('Hamburg', 'Frankfurt', 493), 
         ('Hamburg', 'Stuttgart', 656), 
         ('Hamburg', 'Düsseldorf', 401), 
         ('Munich', 'Cologne', 573), 
         ('Munich', 'Frankfurt', 392), 
         ('Munich', 'Stuttgart', 233), 
         ('Munich', 'Düsseldorf', 612), 
         ('Cologne', 'Frankfurt', 212), 
         ('Cologne', 'Stuttgart', 377), 
         ('Cologne', 'Düsseldorf', 54), 
         ('Frankfurt', 'Stuttgart', 205), 
         ('Frankfurt', 'Düsseldorf', 229), 
         ('Stuttgart', 'Düsseldorf', 416)]
graph.add_weighted_edges_from(edges)

graph_data = nx.to_dict_of_dicts(graph)

Now send a request to the API to create a new optimization instance according to this graph specification. The optimization is stored in the platform and can be referenced with an ID from now on.

In [ ]:
request_body = {
    "instance": {
        "name": "TSP",
        "graph": graph_data
    }
}

response = requests.post(
    url=URL+"/api/optimizations",
    headers=headers,
    json=request_body
)

tsp_id = response.json().get("_id")
f"id: {tsp_id}"

### C. Solve your optimization

Now that you have obtained an ID for your TSP problem, you can start using LunaSolve's other services. In addition to our large library of use cases, we also provide a library of **more than 40 algorithms** that you can choose from to solve your optimization problem.

Similar to the problem formulations, you can also view a list of all solvers that we have implemented and are ready to be used. You can access this list via the `/api/info/solvers/available` endpoint or again via our interactive documentation. You can also get more detailed information about each solver using the `/api/info/solvers` end point.

In [ ]:
response = requests.get(
    url=URL+"/api/info/solvers/available",
    headers=headers
)

response.json()

Each solver represents a different approach to solving optimization problems and therefore can be configured differently. To allow you to configure your algorithm exactly as you wish, we provide the ability to send solver-specific parameters to each run of the algorithms. Again, we provide detailed documentation on all solvers and their corresponding parameters.

Solvers are divided into several categories depending on their origin and/or underlying hardware.

In [ ]:
response = requests.get(
    url=URL+"/api/info/solvers",
    headers=headers
)

solvers = response.json()
solvers.keys()

Let's take a look at all solvers offered by D-Wave Systems.

In [ ]:
dwave_solvers = [solver['full_name'] for solver in solvers['DWave']]
dwave_solvers

In [ ]:
solvers["DWave"]

Now let's assume that we want to solve the problem with Simulated Annealing. Let's first take a look at how this algorithm is defined.

In [ ]:
sa = next(x for x in solvers['DWave'] if x['full_name'] == 'simulated_annealing') 
print(sa['description'])

In case you are not sure which parameters to choose, you can go with our default parameters.

In [ ]:
sa_params = sa['params']
sa_params

Now that we have decided on an algorithm and the parameters, let's actually solve our problem from before.

In [ ]:
response = requests.post(
    url=URL+f"/api/optimizations/{tsp_id}/solutions",
    headers=headers,
    params={"solver_name": "simulated_annealing"},
    # You can also simply omit the json parameter if you want to use the default parameters.
    json=sa_params
)

Since some solvers (especially quantum hardware) can take a long time to call, we don't want to keep you and the execution of your code waiting during this time. Instead, when a solution is requested, a 202 code is returned with a `solution_url` where the final result can be retrieved via the API once the computation is complete.

In [ ]:
solution_url = response.json().get("solution_url")

# Here you may have to wait, depending on the software and hardware you have chosen and how complex your problem is.

response = requests.get(
    url=URL+f"/{solution_url}",
    headers=headers
)

solution = response.json()
solution.get("solution")

### D. Making samples readable

Most optimization problems in our API have built in back-transformations to translate binary solution vectors back into the semantic context of their original problem. You can use this endpoint to quickly make sense of the solutions the solvers generate for you.

In [ ]:
response = requests.get(
    url=URL+f"/api/optimizations/{tsp_id}/representation",
    headers=headers,
    params={"bits": json.dumps(solution.get("solution").get("samples")[0])}
)

response.json().get("solution")

### E. Getting recommendations on solution algorithms

However, in most cases, choosing some arbitrary solver with default parameters is not the most efficient way to solve optimization problems. Instead, you can use LunaSolve's *AI-powered* **Recommendation engine** to get a recommendation for the best solver to solve your particular problem.

*Note: the `/recommendation` endpoint is currently still in a prototypical state and as such will only recommend classical solvers. More to come in future updates.*

In [ ]:
response = requests.get(
    url=URL+f"/api/optimizations/{tsp_id}/recommendation",
    headers=headers
)

recommendation = response.json()
recommendation

Now, use the received recommendation to request a solution from the API

In [ ]:
response = requests.post(
    url=URL+f"/api/optimizations/{tsp_id}/solutions",
    headers=headers,
    params={"solver_name": recommendation.get("solver")},
    json=recommendation.get("solver_params")
)

solution_url = response.json().get("solution_url")

In [ ]:
# Wait for your solution to be complete

response = requests.get(
    url=URL+f"/{solution_url}",
    headers=headers
)

solution = response.json()
solution

### F. Running on quantum hardware

Using the same interface as before for classical optimization heuristics, you can also access real QPU time given you have a valid token from the provider.

First, use our QPU-token API to add a QPU token to your account.

In [ ]:
response = requests.post(
    url=URL+f"/api/users/me/qpu_tokens",
    headers=headers,
    json={
        "name": "dwave-token",
        "provider": "dwave",
        "token": "<YOURTOKENHERE>"
    }
)

In [ ]:
response = requests.post(
    url=URL+f"/api/optimizations/{tsp_id}/solutions",
    headers=headers,
    params={"solver_name": "dwave_qpu"},
)

In [ ]:
response = requests.get(
    url=URL+f"/api/optimizations/{tsp_id}/solutions/648c68ce9f9ac53c2885c9a4",
    headers=headers
)
response.json()

### G. Accessing your optimizations and solutions

As mentioned before, we take care of storing your optimization problems and the corresponding solutions, so you can access them later on again.

In [ ]:
show_limit = 50
response = requests.get(
    url=URL+f"/api/optimizations?limit={show_limit}",
    headers=headers
)
optimizations = response.json()
all_ids = [o['_id'] for o in optimizations]
all_ids

You can now reuse the optimizations you have already created and, for example, get more results from additional solvers.

In [ ]:
opt_id = all_ids[0]

solvers = ['tabu_search', 'simulated_annealing', 'dialectic_search']
for s in solvers:
    response = requests.post(
        url=URL+f"/api/optimizations/{tsp_id}/solutions",
        headers=headers,
        params={"solver_name": s},
    )

And after that, you can similarly access all the solutions you have already created for the problem instance.

In [ ]:
response = requests.get(
    url=URL+f"/api/optimizations/{tsp_id}/solutions",
    headers=headers
)
[sol.get("id") for sol in response.json()]

Finally, if at some point you no longer need or want a particular optimization, you of course have the option to remove it from our platform.

In [ ]:
requests.delete(
    url=URL+f"/api/optimizations/{opt_id}",
    headers=headers
)

## Part 5: Open-ended optimization using manually constructed QUBOs

As useful as the library of optimization problems is, not all scenarios can be fit cleanly into a pre-defined problem definition. Some users may wish to input and solve their own custom mathematical models. 

Our services currently make all the samy functionalities available to optimization problems available to problems formulated as QUBO (quadratic unconstrained binary optimization) matrices. The matrices are passed as nested lists of floats.

### A. Initialize QUBO

First, create a list of lists of floats (or integers) which represents the QUBO in question. Then send that qubo to LunaSolve.

In [ ]:
qubo = [
    [4, 0, 0, 0, -2],
    [0, -2, 0, 0, 0],
    [0, 0, 6, -3, 0],
    [0, 0, -3, 2, 0],
    [-2, 0, 0, 0, 5]
]

In [ ]:
response = requests.post(
    url=URL+f"/api/qubos",
    headers=headers,
    json={"matrix": qubo}
)

qubo_id = response.json().get("_id")
f"id: {qubo_id}"

### B. Solve QUBO

Request the creation of a new solution for the QUBO and retrieve it once solving is completed.

In [ ]:
response = requests.post(
    url=URL+f"/api/qubos/{qubo_id}/solutions",
    headers=headers,
    params={"solver_name": "simulated_annealing"},
    json={"num_sweeps": 100, "num_reads": 10}
)

solution = response.json()
solution

### C. Custom Solving Heuristic

Researchers are often wish to develop their own proprietary heuristics for solving QUBOs effectively. These can involve complicated business logic for pre- and post-processing of results. LunaSolve can support developers of optimization heuristics by unifying the access point to both classical and quantum solvers. 

As an example of how Luna's integration could support the development of an innovative approach to QUBO solving, one can imagine a heuristic which strategically splits a QUBO into sub-QUBOs, which are then sent to existing solvers so that the results can be brought together to for an optimal total solution. This kind of functionality can be easily achieved using LunaSolve, as the business logic behind the novel heuristic can be written and maintained in the desired programming language and corresponding repository, while the Luna platform acts as a signel point of access to the solving heuristics which must be called inside the custom algorithm.

## Part 6: Native quantum circuits

Luna also provides users the ability to send full quantum gate circuits to IBM's Qiskit service for execution. Circuits are encoded according to the Qiskit specification (.py file for source code a .json file containing metadata about the program) and are sent to API in Base64 encoding.

### A. Create circuit

An example circuit and metadata are located in this repository in the `example_circuit` directory.

NOTE: The circuit initializes some qubits, but simply returns the string "Done." upon succesful completion.

In [92]:
import base64

In [32]:
program = None
program_metadata = None

# open the file in binary mode
with open("example_circuit/program.py", 'rb') as file:
    # read the contents of the file
    file_contents = file.read()

    # encode the file contents as base64
    program = base64.b64encode(file_contents)

    # decode to string
    program = program.decode('utf-8')

with open("example_circuit/program.json", "rb") as file:
    file_contents = file.read()
    program_metadata = base64.b64encode(file_contents)
    program_metadata = program_metadata.decode('utf-8')


In [88]:
body = {
    "name": "TestCircuit",
    "circuit": {
        "program": program,
        "program_metadata": program_metadata
    },
    "inputs": {"iterations": 10},
    "options": {"backend_name": "ibmq_lima" }
}
body_json = json.dumps(body)

In [89]:
response = requests.post(
    url=URL+f"/api/circuits",
    headers=headers,
    json=body
)
result = response.json()
id = result.get("_id")
result

{'organization': {'id': '646e2144eb61924ca4b938c2',
  'collection': 'organization'},
 'name': 'TestCircuit',
 'created_date': '2023-07-19T19:36:55.352000',
 'created_by': '646e2144eb61924ca4b938c3',
 'modified_date': '2023-07-19T19:36:56.611000',
 'modified_by': None,
 'read_access': [],
 'write_access': [],
 '_id': '64b83b5752468160e54057cf',
 'result': None}

### B. Get circuit results

This endpoint requests the results of the job from IBM's Qiskit service. Wait times maybe be long for true quantum hardware.

In [93]:
response = requests.get(
    url=URL+f"/api/circuits/{id}",
    headers=headers,
    json=body
)
result = response.json()
result.get("result")

'Queued'